In [2]:
import numpy as np
import tensorflow as tf
import scipy.io
%matplotlib inline

In [3]:
our_images = scipy.io.loadmat('IMAGES_RAW.mat')
images = our_images['IMAGESr']

In [8]:
def random_patch(library, width):
    im = library[:,:,np.random.randint(10)]
    x = np.random.randint(np.shape(im)[0] - width)
    y = np.random.randint(np.shape(im)[1] - width)
    patch = im[x:x+width,y:y+width]
    return patch

def make_batch(library, num, width):
    I = np.zeros((num, width*width))
    count = 0
    while count < num:
        im = random_patch(library, width)
        im_vec = im.reshape(1, width*width)
        I[count] = im_vec
        count = count + 1
    return I

In [168]:
X = make_batch(images, 100, 16)
X = np.transpose(X)
n = np.shape(X)[0]
def f(x):
    return x*x*x
def df(x):
    return 3*x*x
def ddf(x):
    return 6*x
D = tf.placeholder(tf.float32)
T = tf.placeholder(tf.float32)
tee = tf.placeholder(tf.int32)
h = tf.Variable(tf.zeros([256]))
H = tf.transpose(tf.reshape(tf.tile(h, [tee]), [tee, n]))
Jd = tf.Variable(tf.random_normal([n]))
Je = tf.exp(Jd)
J = tf.diag(Je)

In [169]:
score = (1/T)*(-2*tf.reduce_sum(tf.multiply(ddf(D),tf.matmul(J,f(D)))) - tf.reduce_sum(tf.multiply(H,ddf(D)))
-2*tf.reduce_sum(tf.multiply(tf.reshape(tf.tile(tf.diag_part(J),[tee]),[tee,n]), tf.transpose(tf.square(df(D))))) 
+ 2*tf.reduce_sum(tf.multiply(tf.square(df(D)), tf.square(tf.matmul(J,f(D))))) + 
               2*tf.reduce_sum(tf.multiply(tf.square(df(D)),tf.multiply(H,tf.matmul(J,f(D)))))
            + (1/2)*tf.reduce_sum(tf.multiply(tf.square(df(D)), tf.square(H)))) 

In [170]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [171]:
print(sess.run(score, {T:100, D : X, tee : 100}))

7.67311e+07


In [172]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(score)

In [173]:
for i in range(1000):
    sess.run(train, {T:100, D : X, tee : 100})

In [174]:
print(sess.run(score, {T:100, D : X, tee : 100}))

-1.90954
